In [2]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [3]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255
m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 13000
The number of features = 784


In [4]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [5]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [6]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [50] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [7]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 130


In [13]:
#Theta Initialization 
def theta_init(arch=50):
    theta = []
    arch = [arch]
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [14]:
def activation(x):
    return 1/(1+np.exp(-x))

In [15]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    #l1 = activation(np.dot(l0, theta[0]))
    #l2 = activation(np.dot(l1, theta[1]))
    return fm

In [16]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [17]:
theta = theta_init()
fm = forward_prop(X_train, theta)

In [18]:
cost_total(X_train, theta, train_class_enc, m)

4.046812745484527

In [19]:
epochs = []
train_accuracy = []
test_accuracy = []
train_time = []

In [ ]:
for i in range(len(arch_test)):
    theta = theta_init(arch_test[i])
    fm = forward_prop(X_train, theta)
    
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_train, theta, train_class_enc, m)

    while(True):
        count = 0

        print("Initial Cost for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
        epoch+=1 #Number of epochs
        cost_final = cost_total(X_train, theta, train_class_enc, m)
        print("Cost after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-06):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))

Initial Cost for this epoch 0 = 3.782566581985748
Error = 3.793485897465157
Error = 3.0613402403896517
Error = 2.034884942839799
Cost after 1 epochs is = 1.885709698317633
Initial Cost for this epoch 1 = 1.885709698317633
Error = 1.9036657585988124
Error = 1.2513109017083102
Error = 0.9860623060682582
Cost after 2 epochs is = 0.935702134554877
Initial Cost for this epoch 2 = 0.935702134554877
Error = 0.9526947724135306
Error = 0.7991277139208917
Error = 0.7371881982286652
Cost after 3 epochs is = 0.7127481353220118
Initial Cost for this epoch 3 = 0.7127481353220118
Error = 0.7206854922055217
Error = 0.6625859301661476
Error = 0.6419224275706084
Cost after 4 epochs is = 0.6260851262343319
Initial Cost for this epoch 4 = 0.6260851262343319
Error = 0.6292757420133975
Error = 0.6013277901470186
Error = 0.5938236658807474
Cost after 5 epochs is = 0.5821250544993088
Initial Cost for this epoch 5 = 0.5821250544993088
Error = 0.5835566923069859
Error = 0.5677345201989872
Error = 0.565428543483

Cost after 47 epochs is = 0.48216485699747486
Initial Cost for this epoch 47 = 0.48216485699747486
Error = 0.4821463833731729
Error = 0.4821004230755114
Error = 0.48256824283081623
Cost after 48 epochs is = 0.48208971505994147
Initial Cost for this epoch 48 = 0.48208971505994147
Error = 0.4820716580092872
Error = 0.48202859262206743
Error = 0.48248488180195054
Cost after 49 epochs is = 0.482019386962817
Initial Cost for this epoch 49 = 0.482019386962817
Error = 0.4820017307507287
Error = 0.48196134756753256
Error = 0.48240661669093005
Cost after 50 epochs is = 0.48195350254814984
Initial Cost for this epoch 50 = 0.48195350254814984
Error = 0.48193623218959786
Error = 0.481898337622722
Error = 0.482333066198282
Cost after 51 epochs is = 0.48189172559714955
Initial Cost for this epoch 51 = 0.48189172559714955
Error = 0.4818748268215209
Error = 0.48183924436639514
Error = 0.48226388324889213
Cost after 52 epochs is = 0.4818337501849773
Initial Cost for this epoch 52 = 0.4818337501849773
E

Error = 0.4809293962111789
Error = 0.4811429948000418
Cost after 93 epochs is = 0.48093370921624196
Initial Cost for this epoch 93 = 0.48093370921624196
Error = 0.480925190277957
Error = 0.4809229220565041
Error = 0.4811339422184097
Cost after 94 epochs is = 0.4809272007029318
Initial Cost for this epoch 94 = 0.4809272007029318
Error = 0.480918786692442
Error = 0.48091673608716906
Error = 0.481125240926536
Cost after 95 epochs is = 0.48092097787682164
Initial Cost for this epoch 95 = 0.48092097787682164
Error = 0.4809126664139036
Error = 0.4809108242670256
Error = 0.48111687472706677
Cost after 96 epochs is = 0.48091502693209354
Initial Cost for this epoch 96 = 0.48091502693209354
Error = 0.4809068157123314
Error = 0.4809051733222425
Error = 0.48110882827903567
Cost after 97 epochs is = 0.48090933480835113
Initial Cost for this epoch 97 = 0.48090933480835113
Error = 0.48090122160005666
Error = 0.48089977069464385
Error = 0.481101087046236
Cost after 98 epochs is = 0.4809038891452338
In

Cost after 140 epochs is = 0.480799059346059
cost initial= 0.4808000323359165 , cost final=0.480799059346059 , change in cost= -9.72989857517259e-07
Initial Cost for this epoch 0 = 3.804659420288236
Error = 3.8520046874470335
Error = 1.6220880162081635
Error = 0.7747159824815721
Cost after 1 epochs is = 0.7324106625227561
Initial Cost for this epoch 1 = 0.7324106625227561
Error = 0.7189631450525299
Error = 0.6009277741418297
Error = 0.5477388851719889
Cost after 2 epochs is = 0.5420676902456073
Initial Cost for this epoch 2 = 0.5420676902456073
Error = 0.5389115187980774
Error = 0.5231273293159173
Error = 0.5122359747004076
Cost after 3 epochs is = 0.5088053158868209
Initial Cost for this epoch 3 = 0.5088053158868209
Error = 0.5060210858271168
Error = 0.5027348340493105
Error = 0.4995981624489397
Cost after 4 epochs is = 0.4968729468045385
Initial Cost for this epoch 4 = 0.4968729468045385
Error = 0.49415079330082967
Error = 0.494214211681194
Error = 0.49336395965835095
Cost after 5 ep

Error = 0.4799136724637104
Cost after 47 epochs is = 0.4800635296693141
Initial Cost for this epoch 47 = 0.4800635296693141
Error = 0.47855221872502784
Error = 0.47981402499101033
Error = 0.4798634175387255
Cost after 48 epochs is = 0.48002691821030785
Initial Cost for this epoch 48 = 0.48002691821030785
Error = 0.4785146659583317
Error = 0.47977292316100867
Error = 0.47981198239782064
Cost after 49 epochs is = 0.4799892110258308
Initial Cost for this epoch 49 = 0.4799892110258308
Error = 0.4784754400303831
Error = 0.4797314145773185
Error = 0.47975928554903535
Cost after 50 epochs is = 0.4799503608806791
Initial Cost for this epoch 50 = 0.4799503608806791
Error = 0.4784343971001252
Error = 0.4796894613079576
Error = 0.4797052418588581
Cost after 51 epochs is = 0.4799103230480769
Initial Cost for this epoch 51 = 0.4799103230480769
Error = 0.4783913841210968
Error = 0.4796470155413383
Error = 0.4796497604213292
Cost after 52 epochs is = 0.479869054723971
Initial Cost for this epoch 52 =

...

Error = 0.3800757523104221
Error = 0.4002464434378697
Cost after 7839 epochs is = 0.3922822318928649
Initial Cost for this epoch 7839 = 0.3922822318928649
Error = 0.3774131108294622
Error = 0.38007519132804596
Error = 0.40024023040732404
Cost after 7840 epochs is = 0.3922782271243166
Initial Cost for this epoch 7840 = 0.3922782271243166
Error = 0.377411426729913
Error = 0.38007464008407055
Error = 0.40023399255778896
Cost after 7841 epochs is = 0.3922742105440021
Initial Cost for this epoch 7841 = 0.3922742105440021
Error = 0.3774097507385446
Error = 0.38007409863365416
Error = 0.4002277293665546
Cost after 7842 epochs is = 0.3922701819532297
Initial Cost for this epoch 7842 = 0.3922701819532297
Error = 0.37740808275037674
Error = 0.38007356703078926
Error = 0.40022144029977996
Cost after 7843 epochs is = 0.3922661411494604
Initial Cost for this epoch 7843 = 0.3922661411494604
Error = 0.3774064226668197
Error = 0.38007304532823794
Error = 0.40021512481222987
Cost after 7844 epochs is =

Error = 0.37734455775765813
Error = 0.38006053969108716
Error = 0.39992477769753876
Cost after 7885 epochs is = 0.39208198709512765
Initial Cost for this epoch 7885 = 0.39208198709512765
Error = 0.3773431921185393
Error = 0.38006044858955307
Error = 0.3999166603829441
Cost after 7886 epochs is = 0.39207715548062777
Initial Cost for this epoch 7886 = 0.39207715548062777
Error = 0.3773418327516982
Error = 0.3800603666865702
Error = 0.39990847756101816
Cost after 7887 epochs is = 0.39207229699926016
Initial Cost for this epoch 7887 = 0.39207229699926016
Error = 0.3773404795592961
Error = 0.3800602938226747
Error = 0.3999002278263396
Cost after 7888 epochs is = 0.39206741112897
Initial Cost for this epoch 7888 = 0.39206741112897
Error = 0.3773391324321173
Error = 0.3800602298260598
Error = 0.3998919097449978
Cost after 7889 epochs is = 0.3920624973360467
Initial Cost for this epoch 7889 = 0.3920624973360467
Error = 0.3773377912486444
Error = 0.3800601745118954
Error = 0.3998835218543637
Co

Error = 0.38005989776571014
Error = 0.39945815220344283
Cost after 7931 epochs is = 0.39182190496439173
Initial Cost for this epoch 7931 = 0.39182190496439173
Error = 0.3772814885781288
Error = 0.3800596741955911
Error = 0.39944524251789704
Cost after 7932 epochs is = 0.3918151060779024
Initial Cost for this epoch 7932 = 0.3918151060779024
Error = 0.37727987287689707
Error = 0.3800594181153619
Error = 0.39943219025373977
Cost after 7933 epochs is = 0.39180824459307567
Initial Cost for this epoch 7933 = 0.39180824459307567
Error = 0.37727822166332275
Error = 0.38005912709122913
Error = 0.39941899482746396
Cost after 7934 epochs is = 0.39180131973915694
Initial Cost for this epoch 7934 = 0.39180131973915694
Error = 0.3772765325938219
Error = 0.38005879860285446
Error = 0.3994056558243939
Cost after 7935 epochs is = 0.3917943307809392
Initial Cost for this epoch 7935 = 0.3917943307809392
Error = 0.37727480326715096
Error = 0.3800584300468512
Error = 0.39939217300925045
Cost after 7936 epo

Error = 0.3799761667031134
Error = 0.3987223486266702
Cost after 7978 epochs is = 0.39143431332505974
Initial Cost for this epoch 7978 = 0.39143431332505974
Error = 0.37713745594380543
Error = 0.37997248499179775
Error = 0.39870491124592305
Cost after 7979 epochs is = 0.3914248727401318
Initial Cost for this epoch 7979 = 0.3914248727401318
Error = 0.3771328295532156
Error = 0.37996875927471363
Error = 0.39868743486245484
Cost after 7980 epochs is = 0.3914154037148426
Initial Cost for this epoch 7980 = 0.3914154037148426
Error = 0.37712816909031277
Error = 0.3799649943600335
Error = 0.3986699184461577
Cost after 7981 epochs is = 0.3914059070605521
Initial Cost for this epoch 7981 = 0.3914059070605521
Error = 0.3771234772269497
Error = 0.3799611950593153
Error = 0.3986523606492711
Cost after 7982 epochs is = 0.3913963834717032
Initial Cost for this epoch 7982 = 0.3913963834717032
Error = 0.3771187565071888
Error = 0.3799573661571559
Error = 0.398634759826346
Cost after 7983 epochs is = 0

Error = 0.39786525093269187
Cost after 8023 epochs is = 0.39098019646703003
Initial Cost for this epoch 8023 = 0.39098019646703003
Error = 0.3769154446559496
Error = 0.3798113332791958
Error = 0.39784377469613263
Cost after 8024 epochs is = 0.39096923480687196
Initial Cost for this epoch 8024 = 0.39096923480687196
Error = 0.37691037278709444
Error = 0.3798083039784368
Error = 0.3978221752349038
Cost after 8025 epochs is = 0.3909582250074591
Initial Cost for this epoch 8025 = 0.3909582250074591
Error = 0.3769052952799548
Error = 0.3798052900579023
Error = 0.3978004524745763
Cost after 8026 epochs is = 0.3909471664923222
Initial Cost for this epoch 8026 = 0.3909471664923222
Error = 0.37690021177874067
Error = 0.37980228994016335
Error = 0.3977786063805515
Cost after 8027 epochs is = 0.3909360586924057
Initial Cost for this epoch 8027 = 0.3909360586924057
Error = 0.3768951218905633
Error = 0.37979930202050255
Error = 0.3977566369527319
Cost after 8028 epochs is = 0.39092490104531036
Initi

In [22]:
print("The total training time = {} sec and epochs = {}".format(time.time()-start,epoch))

The total training time = 630.740788936615 sec and epochs = 2967


In [16]:
print("The total training time = {} sec and epochs = {}".format(time.time()-start,epoch))

The total training time = 2063.004090309143 sec and epochs = 16601


In [28]:
epochs= []
print(arch_test)

[1, 5, 10, 50, 100]


In [22]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [37]:
calc_accuracy(X_test, theta, test_actual_class_enc)

2.6615384615384614